### Farthest Point Sampling 

Using PyMesh to load the obj files. Sample around 10,000 points randomly from the mesh and choose 1000 points using farthest point sampling.

In [ ]:
import pymesh
import numpy as np

In [ ]:
teapot = pymesh.load_mesh("teapot.obj")
violin_case = pymesh.load_mesh("violin_case.obj")

In [ ]:
def calculate_triangle_area(v):
    a = np.linalg.norm(v[0]-v[1])
    b = np.linalg.norm(v[0]-v[2])
    c = np.linalg.norm(v[1]-v[2])
    s = (a+b+c)/2.
    area = (s*(s-a)*(s-b)*(s-c))**0.5
    return area

In [ ]:
triangle_areas = np.zeros(violin_case.num_faces)
for i in range(violin_case.num_faces):
    vidx = violin_case.faces[i]
    vertices = violin_case.vertices[vidx,:]
    triangle_areas[i] = calculate_triangle_area(vertices)

weights = triangle_areas/np.sum(triangle_areas)
num_points = np.round(weights*10000).astype(np.int)

P = np.zeros([sum(num_points),3])

begin = 0
for i in range(violin_case.num_faces):
    r1 = np.random.uniform(0,1,num_points[i]).reshape([num_points[i],1])
    r2 = np.random.uniform(0,1,num_points[i]).reshape([num_points[i],1])
    
    A = np.zeros([num_points[i],3])
    A[:,] = violin_case.vertices[violin_case.faces[i][0],:]
    
    B = np.zeros([num_points[i],3])
    B[:,] = violin_case.vertices[violin_case.faces[i][1],:]
    
    C = np.zeros([num_points[i],3])
    C[:,] = violin_case.vertices[violin_case.faces[i][2],:]
    
    D = np.multiply(1-np.sqrt(r1),A)\
        + np.multiply(np.multiply(np.sqrt(r1),1-r2),B)\
        + np.multiply(np.multiply(np.sqrt(r1),r2),C)
    
    P[begin:begin+num_points[i],:] = D
    begin += num_points[i]

S = np.zeros([1000,3])
idx = np.random.randint(sum(num_points),size=1)
S[0,:] = P[idx,:]
n = 1
indices = idx.tolist()

while n < 1000:
    dist = np.zeros(sum(num_points))
    for i in range(sum(num_points)):
        Q = np.zeros([n,3])
        Q[:,] = P[i,:]
        D = np.linalg.norm(Q-S[:n,:],axis=1)
        dist[i] = min(D)
    sort_dist = np.argsort(dist)[::-1]
#     sort_dist = np.delete(sort_dist,[np.where(indices[k]==sort_dist)[0].tolist()[0] for k in range(n)])
    indices.append(sort_dist[0])
    print(sort_dist[0])
    S[n,:] = P[indices[-1],:]
    n += 1

In [ ]:
new_mesh = pymesh.tetgen()
new_mesh.points = S
new_mesh.run()
sampled_violin_case = new_mesh.mesh
pymesh.save_mesh("sampled_violin_case.obj", sampled_violin_case, use_float=True)